# Experiment 004: Use chistyakov_best.csv as Baseline

The evaluator identified that our overlap validation was wrong. The LB uses:
- `intersects() and not touches()` for overlap detection
- Scaled coordinates (1e15)

chistyakov_best.csv has 0 overlaps and should pass LB validation.

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import warnings
warnings.filterwarnings('ignore')

# Set precision for Decimal (matching LB)
getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Libraries loaded')

In [ ]:
# Tree geometry constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    return Polygon(translated)

def get_bounding_box_side(polygons):
    if not polygons:
        return 0.0
    union = unary_union(polygons)
    bounds = union.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

def parse_submission(df):
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

print('Helper functions defined')

In [ ]:
# CORRECT overlap check - matching LB exactly
# LB uses: intersects() and not touches()
def check_lb_overlaps(polygons):
    """Check for overlaps using the EXACT LB method: intersects() and not touches()."""
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return True, (i, j)
    return False, None

print('LB overlap check defined')

In [ ]:
# Load chistyakov_best.csv
print('Loading chistyakov_best.csv...')
chistyakov_df = pd.read_csv('/home/code/preoptimized/chistyakov_best.csv')
print(f'Shape: {chistyakov_df.shape}')
print(chistyakov_df.head())

chistyakov_parsed = parse_submission(chistyakov_df)

In [ ]:
# Calculate score
def calculate_score(df):
    scores = {}
    total_score = 0.0
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        config_df = df[df['id'].str.startswith(prefix)]
        if len(config_df) != n:
            print(f'Warning: Config {n} has {len(config_df)} trees')
            continue
        polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
        side = get_bounding_box_side(polygons)
        scores[n] = side
        total_score += (side ** 2) / n
    return total_score, scores

print('Calculating chistyakov score...')
chistyakov_score, chistyakov_sides = calculate_score(chistyakov_parsed)
print(f'Score: {chistyakov_score:.6f}')

In [ ]:
# Check for LB overlaps
print('Checking for LB overlaps (intersects and not touches)...')
overlap_configs = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    has_overlap, pair = check_lb_overlaps(polygons)
    if has_overlap:
        overlap_configs.append(n)

print(f'Configs with LB overlaps: {len(overlap_configs)}')
if overlap_configs:
    print(f'Overlap configs: {overlap_configs}')
else:
    print('NO OVERLAPS! chistyakov_best.csv should pass LB validation.')

In [ ]:
# Also check minimum distance between trees to understand separation
print('\nChecking minimum distances between trees...')

def get_min_distance(polygons):
    """Get minimum distance between any two polygons."""
    min_dist = float('inf')
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            dist = polygons[i].distance(polygons[j])
            if dist < min_dist:
                min_dist = dist
    return min_dist

# Check a few configs
for n in [5, 10, 50, 100, 200]:
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    min_dist = get_min_distance(polygons)
    print(f'N={n}: min_dist = {min_dist:.2e}')

In [ ]:
# Copy chistyakov_best.csv as our submission
import os
import shutil

os.makedirs('/home/submission', exist_ok=True)
shutil.copy('/home/code/preoptimized/chistyakov_best.csv', '/home/submission/submission.csv')
shutil.copy('/home/code/preoptimized/chistyakov_best.csv', '/home/code/experiments/004_chistyakov_baseline/submission.csv')

print('Submission saved!')
print(f'Score: {chistyakov_score:.6f}')
print(f'Target: 68.896973')
print(f'Gap: {chistyakov_score - 68.896973:.6f}')

In [ ]:
# Summary
print('\n' + '='*60)
print('EXPERIMENT 004 SUMMARY')
print('='*60)
print(f'Using: chistyakov_best.csv')
print(f'Score: {chistyakov_score:.6f}')
print(f'LB overlaps: {len(overlap_configs)}')
print(f'Target: 68.896973')
print(f'Gap: {chistyakov_score - 68.896973:.6f}')
print('='*60)
print('\nThis submission should pass LB validation.')
print('If it passes, we can create a hybrid to improve score.')